# 量子テレポーテーション

このノートでは量子テレポーテーションについて説明します。まず始めに、作成した量子回路を Qiskit に組み込まれているシミュレーターを使ってテストします。その後、その回路を実際の量子コンピューターで動かしてみます。

## 目次

1. [概要](#overview)    
2. [量子テレポーテーション・プロトコル](#how)       
3. [テレポーテーション・プロトコルのシミュレーション](#simulating)   
    3.1 [実際の量子コンピュータにおける結果の検査方法](#testing)   
    3.2 [状態ベクトル・シミュレーターを使った方法](#simulating-sv)     
    3.3 [QASM・シミュレーターを使った方法](#simulating-qs)    
4. [実際の量子コンピュータにおけるテレポーテーション]](#real_qc)    
    4.1 [IBM ハードウェアと遅延測定](#deferred-measurement)    
    4.2 [実行](#executing)    
4. [参考文献](#references)

## 1. 概要 <a id='overview'></a>

アリスがボブに量子情報を送りたいとしましょう。これを具体的に、アリスがボブに状態
$\vert\psi\rangle = \alpha\vert0\rangle + \beta\vert1\rangle$
を送りたい、と仮定します。その為にはまず、$\alpha$ と $\beta$ の情報をボブに渡す必要があります。

量子力学には、未知の量子状態の単純な複製を正確に作ることは出来ない、という定理が存在します。量子複製不可能定理（no-cloning theorem）として知られているものです。この定理ゆえ、アリスが単純に $\vert\psi\rangle$ の複製を生成してそれをボブに渡す、という事は出来ません。状態 （係数ではありません）の複製は古典的な計算でのみ可能なのです。

しかし、2つの古典的ビットとエンタングルメントを利用することで、アリスは状態 $\vert\psi\rangle$ をボブに転送できます。その結果、最終的にボブが $\vert\psi\rangle$ を持ち、アリスがもはや何も持っていない事から、我々はこれをテレポーテーションと呼んでいます。これがどのような仕組みから成るのか、具体例と共に見てみましょう。

## 2. 量子テレポーテーション・プロトコル <a id='how'></a>
量子ビットを転送するためには、アリスとボブは第三者（ここではイブと呼びましょう）を使ってエンタングルした量子ビットのペアを作り、それを彼ら自身に送ってもらわなくてはなりません。その後アリスは、彼女の量子ビットにいくつかの演算子を作用させ、その結果を古典通信チャンネルを通してボブに送信します。ボブはそれに従って彼の量子ビットにいくつか演算子を作用させることで、結果的にアリスの量子ビットを受け取ることが出来ます。

![teleportation_doodle](images/tele1.png)

これらの手順を、以下では量子回路を使って説明していきます。ここでは実際に量子ビットを「送る」わけではありませんので、想像しながら進んでください！

まず初めに、セッション（Jupyter Noteobook上の対話型スクリプトの事）をセットアップしていきます：

In [ ]:
# 必要なモジュールをインポートします
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
# Jupyter Notebooks 上できれいに画像を表示させるための設定をします。
%config InlineBackend.figure_format = 'svg'

それでは量子回路を作成していきます：

In [ ]:
qr = QuantumRegister(3)    # プロトコルでは、３つの量子ビットと、
crz = ClassicalRegister(1) # ２つの異なるレジスタに登録した
crx = ClassicalRegister(1) # ２つの古典ビットを使用します。
teleportation_circuit = QuantumCircuit(qr, crz, crx)

#### Step 1
第三者のイブは、エンタングルした量子ビットのペアを作り、片方をボブへ、もう片方をアリスへ渡します。

イブが作ったこのペアは Bell pair（ベルペア）と呼ばれる特別なものです。量子回路の言葉を使って Bell pair の作成方法を説明すると、まず量子ビットのペアのうち１つをアダマールゲートで X 基底 ($|+\rangle$ and $|-\rangle$) に変換し、これをコントロール・ビットとして、もう片方の量子ビットに CNOT ゲートを適用することで作成できます。

In [ ]:
def create_bell_pair(qc, a, b):
    """Creates a bell pair in qc using qubits a & b"""
    qc.h(a) # Put qubit a into state |+>
    qc.cx(a,b) # CNOT with a as control and b as target

# In our case, Eve entangles qubits q1 and q2
# Let's apply this to our circuit:
create_bell_pair(teleportation_circuit, 1, 2)
# And view the circuit so far:
teleportation_circuit.draw('mpl')

以下では、アリスは $q_1$、ボブは $q_2$ を持っているとしましょう。

#### Step 2 

アリスは、コントロール・ビットを $\vert\psi\rangle$ （彼女がボブに送ろうとしている量子ビット）として、$q_1$ に CNOT ゲートを作用させます。そして、$|\psi\rangle$ にアダマールゲートを作用させます。今回の量子回路では、アリスが送信しようとしている量子ビット（$|\psi\rangle$）は $q_0$ です：

In [ ]:
def alice_gates(qc, psi, a):
    qc.cx(psi, a)
    qc.h(psi)
    
# 関数を回路に組み込みましょう：
teleportation_circuit.barrier() # ステップを分割するために「バリア」を使います
alice_gates(teleportation_circuit, 0, 1)
teleportation_circuit.draw('mpl')

#### Step 3

次にアリスは、彼女が持っている両方の量子ビット $q_1$ と $\vert\psi\rangle$ を測定し、その結果を２つの古典ビットに格納します。これら２つのビットを、彼女はボブに送信します。

In [ ]:
def measure_and_send(qc, a, b):
    """量子ビット a と b 測定し、結果をボブに「送信」します"""
    qc.barrier()
    qc.measure(a,0)
    qc.measure(b,1)

measure_and_send(teleportation_circuit, 0 ,1)
teleportation_circuit.draw('mpl')

**Step 4**: 既に $q_2$ を持っているボブは、送られてきた古典ビットの状態に従って $q_2$  に次のゲートを作用させます:

00 $\rightarrow$ 何もしません

01 $\rightarrow$ $X$ ゲートを適用

10 $\rightarrow$ $Z$ ゲートを適用

11 $\rightarrow$ $ZX$ ゲートを適用

(*注：情報の伝達は純粋に古典的な方法で行われます。*.)

In [ ]:
# この関数は、作用させるゲートを決定するために、
# 量子回路（qc : QuantumCircuit）、整数（qubit）、
# 古典レジスタ（crz & crx : ClassicalRegisters）を引数に持ちます
def bob_gates(qc, qubit, crz, crx):
    # 個々では、量子ビットの代わりに古典ビットでゲート操作する為に
    # c_if を用います
    qc.z(qubit).c_if(crz, 1) # レジスタが１の場合に
    qc.x(qubit).c_if(crx, 1) # ゲートを作用させます

In [ ]:
# 関数を回路に組み込みましょう：
teleportation_circuit.barrier() # ステップを分割するために「バリア」を使います
bob_gates(teleportation_circuit, 2, crz, crx)
teleportation_circuit.draw('mpl')

できました！プロトコルの最後に、アリスの量子ビットはボブにテレポートされます。

## 3. テレポーテーション・プロトコルのシミュレーション <a id='simulating'></a>

### 3.1 実際の量子コンピューターにおける結果の検査方法 <a id='testing'></a>

この notebook では、アリスの量子ビットをランダム状態 $\vert\psi\rangle$ （「psi」）で初期化します。この状態は「初期化」ゲートを $|q_0\rangle$ に作用させる事で作られます。この章では「psi」を選ぶために「ランダム状態」関数を使いますが、「psi」を任意の量子ビット状態に設定してもかまいません。

In [ ]:
from qiskit_textbook.tools import random_state, vector2latex
# ランダムな１量子ビット状態を作ります
psi = random_state(1)

# 表示を綺麗にします
vector2latex(psi, pretext="|\\psi\\rangle =")
# Bloch球で表示します
plot_bloch_multivector(psi)

$|0\rangle$ から $|\psi\rangle$ を作る初期化ゲートを作りましょう：

In [ ]:
from qiskit.extensions import Initialize
init_gate = Initialize(psi)

量子テレポーテーション回路が動作すれば、回路の最後に量子ビット $|q_2\rangle$ を状態がこの状態になるはずです。状態ベクトルシミュレーターを使ってこれをチェックしてみましょう。

### 3.2 状態ベクトル・シミュレーターを使った方法 <a id='simulating-sv'></a>

状態ベクトルシミュレーターを使うと、量子ビットがテレポートされたかを確かめることができます。

In [ ]:
qr = QuantumRegister(3)   # プロトコルでは３量子ビットと
crz = ClassicalRegister(1) # ２つの古典レジスタを使います
crx = ClassicalRegister(1)
qc = QuantumCircuit(qr, crz, crx)

# 初めに、アリスの q0 を初期化しましょう
qc.append(init_gate, [0])
qc.barrier()

# テレポーテーション・プロトコルを開始します
create_bell_pair(qc, 1, 2)
qc.barrier()
# q1 をアリスに、q2 をボブに渡します
alice_gates(qc, 0, 1)

# その後アリスが彼女の古典ビットをボブに送信します
measure_and_send(qc, 0, 1)

# ボブが量子ビットを復号します
bob_gates(qc, 2, crz, crx)

qc.draw('mpl')

この記事の執筆時点では、上の画像の「初期化」ゲートにレンダリングの問題がありますが、回路は正常に動作しています。以下でみるように、この状態ベクトルシミュレーターを用いると、状態$|q_2\rangle$ は上で作った状態 $|\psi\rangle$ と同じですが、状態 $|q_0\rangle$ と $|q_1\rangle$ は、状態 $|0\rangle$ か $|1\rangle$ のどちらかに崩れている事がわかります。状態 $|\psi\rangle$ は 量子ビット 0 から 2 にテレポートされています。

In [ ]:
backend = Aer.get_backend('statevector_simulator')
out_vector = execute(qc, backend).result().get_statevector()
plot_bloch_multivector(out_vector)

このセルを何回か実行してみてください。すると、量子ビット 0 と 1 の状態が変化している事に気づくと思いますが、量子ビット 2 は常に状態 $|\psi\rangle$ のままです。

### 3.3 QASM シミュレーターを使った方法 <a id='simulating-qs'></a>

Quantum teleportation is designed to send qubits between two parties. We do not have the hardware to demonstrate this, but we can demonstrate that the gates perform the correct transformations on a single quantum chip. Here we use the QASM simulator to simulate how we might test our protocol.

量子テレポーテーションは２つのグループ間で量子ビットを送るようにデザインされています。これを実証するハードウェアを私たちは持ち合わせていませんが、単一の量子チップ上でなら、ゲートが正しい変換を行う事を実証することができます。ここでは、QASM シミュレーターを使用して、プロトコルをテストする方法をシミュレートします。

実際の量子コンピューターでは、状態ベクトルをサンプリングする事はできません。なので、テレポーテーション回路が稼働しているかをチェックしたければ手段を変える必要があります。みなさんは、量子ビット $|0\rangle$　を状態 $|\psi\rangle$ に変えるために初期化したことを覚えているでしょう：

$$ |0\rangle \xrightarrow{\text{初期化}} |\psi\rangle $$

全ての量子ゲートは可逆性を持っているので、次のようにして初期化の逆変換を得ることが出来ます：

In [ ]:
inverse_init_gate = init_gate.gates_to_uncompute()

この演算子は次のような性質を持っています：

$$ |\psi\rangle \xrightarrow{\text{逆初期化}} |0\rangle $$

量子ビット $|q_0\rangle$ が $|q_2\rangle$ にテレポートした事を証明するには、$|q_2\rangle$ に対してこの逆初期化を行えば確実に $|0\rangle$ が測定できる事を期待すればよいのです。下の回路ではこれを実行しています：

In [ ]:
qc.append(inverse_init_gate, [2])
qc.draw('mpl')

ここでも、「inverse_init_gate」（回路図では「disentangler」とよばれています）のレンダリングに問題がありますが、画像の中にゲートが表れているのがはっきりとわかります。最後に、３番目の古典ビットを測定して出力を確認します：

In [ ]:
# 結果を見るためには
# 新しい古典レジスタを追加する必要があります
cr_result = ClassicalRegister(1)
qc.add_register(cr_result)
qc.measure(2,2)
qc.draw('mpl')

それでは検査を実行しましょう：

In [ ]:
backend = Aer.get_backend('qasm_simulator')
counts = execute(qc, backend, shots=1024).result().get_counts()
plot_histogram(counts)

q_2$ （文字列の左端のビット）の測定で 状態 $|0\rangle$ を得る確率が 100% であることがわかります。これは予想された結果であり、テレポーテーション・プロトコルが正常に動作した事を示しています。

## 3. 実際の量子コンピューターにおけるテレポーテーション <a id='real_qc'></a>

### 3.1 IBMハードウェアと遅延測定 <a id='deferred-measurement'></a>

IBMの量子コンピュータは現在、測定の後に行う命令系をサポートしていないため、実際のハードウェア上ではこれまでの形の回路による量子テレポーテーションを実行できません。ただ幸いなことに、「遅延測定原理」[1]のおかげで、この事態が私たちの計算実行能力を制限することはありません。この原理は、どのような測定も回路の終端まで延期することができるというものです。つまりすべての測定を最後に持ってくる事が可能、そこで測定しても同じ結果が得られるはずなのです。

![deferred_measurement_gates](images/defer_measurement.svg)

早い段階で測定を行うことで得られるメリットは、ハードウェアと関連しています。早期に測定できれば、量子ビットを再利用できるかもしれませんし、量子ビットが壊れやすい重ね合わせ状態にある時間を短縮できるかもしれません。この例では、量子テレポーテーションにおける早期測定が、直接量子通信チャンネルを使わず量子ビットの状態を送信出来ることを見ます。

ゲートを動かすことで実際のハードウェア上で「テレポーテーション」回路を実証することができる一方で、テレポーテーションプロセスの恩恵の一部が失われていることに注意してください。

それでは、`bob_gates`関数を次のように書き直しましょう：

In [ ]:
def bob_gates(qc, a, b, c):
    qc.cz(a, c)
    qc.cx(b, c)

そして新しい回路を作ります：

In [ ]:
qc = QuantumCircuit(3,1)

# 初めにアリスの q0 を初期化しましょう
qc.append(init_gate, [0])
qc.barrier()

# テレポーテーション・プロトコルを開始します
create_bell_pair(qc, 1, 2)
qc.barrier()
# q1 をアリスに、q2 をボブに渡します
alice_gates(qc, 0, 1)
qc.barrier()
# アリスが彼女の古典ビットをボブに送信します
bob_gates(qc, 0, 1, 2)

# 初期化プロセスを元に戻します
qc.append(inverse_init_gate, [2])

# ２番目の量子ビットの状態のみに注目して結果を見ます
qc.measure(2,0)

# 結果を表示します
qc.draw(output='mpl')

### 4.2 実行 <a id='executing'></a> 

In [10]:
# まず初めに、保存したアカウントをロードして、使用できるデバイスを確認します
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [ ]:
# IBMで最も空いているバックエンドを取得し、量子回路を走らせます
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(filters=lambda b: b.configuration().n_qubits >= 3 and
                                   not b.configuration().simulator and b.status().operational==True))
job_exp = execute(qc, backend=backend, shots=8192)

In [ ]:
# 結果を取得・表示します
exp_result = job_exp.result()
exp_measurement_result = exp_result.get_counts(qc)
print(exp_measurement_result)
plot_histogram(exp_measurement_result)

このように、$|1\rangle$ の測定結果は複数あります。これらはゲートと量子ビットのエラーに起因したものです。対照的に、この notebook のはじめに使ったシミュレーターではゲートのエラーはゼロであり、エラーのないテレポーテーションができました。

In [12]:
error_rate_percent = sum([exp_measurement_result[result] for result in exp_measurement_result.keys() if result[0]=='1']) \
                    * 100./ sum(list(exp_measurement_result.values()))
print("The experimental error rate : ", error_rate_percent, "%")

The experimental error rate :  60.0341796875 %


## 5. 参考文献 <a id='references'></a>
[1] M. Nielsen and I. Chuang, Quantum Computation and Quantum Information, Cambridge Series on Information and the Natural Sciences (Cambridge University Press, Cambridge, 2000).

In [13]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.12.0',
 'qiskit-aer': '0.4.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.4.6',
 'qiskit-aqua': '0.6.4',
 'qiskit': '0.15.0'}